### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []
for page_num in range(1, 350):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/linh-vuc-khac/t1/16/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
 

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_linhvuckhac = pd.DataFrame(articles_data)
df_linhvuckhac.head(5)

,Name,Time,Question,Situation,Answer
0,Đào Long Sơn,14:49 28/08/2023,Thi tìm hiểu pháp luật có phải hoạt động văn h...,"Tôi xin hỏi, các địa phương tổ chức các cuộc t...","Bộ Văn hóa, Thể thao và Du lịch trả lời vấn đề..."
1,Nguyên Hạnh,23:37 28/06/2023,Tổ chức cuộc thi về ẩm thực có cần xin giấy phép?,Đơn vị tôi muốn tổ chức một cuộc thi về ẩm thự...,"Cục Văn hóa cơ sở, Bộ Văn hóa, Thể thao và Du ..."
2,Nguyễn Văn Tú,20:58 24/05/2023,Có quy định xét công nhận danh hiệu 'Tộc họ vă...,"Tôi muốn đăng ký xây dựng ""Tộc họ văn hoá"" của...","Cục Văn hóa cơ sở, Bộ Văn hóa, Thể thao và Du ..."
3,Đỗ Quốc Hưng,07:05 18/03/2023,"Có cần phần ""Kính gửi"" trong báo cáo không?","Nghị định số 30/2020/NĐ-CP quy định, ngoài tờ ...",Bộ Nội vụ trả lời vấn đề này như sau:Báo cáo l...
4,Lý Hoàng Long,09:34 17/03/2023,Tổ chức biểu diễn trên mạng có cần xin cấp phép?,Tôi dự định mở một cuộc biểu diễn trên nền tản...,"Cục Nghệ thuật biểu diễn, Bộ Văn hóa, Thể thao..."


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../data/data_tinhhuong/linhvuckhac_data.csv'
df_linhvuckhac.to_csv(csv_file, encoding='utf-8',index_label='Index')
print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/data_tinhhuong/linhvuckhac.csv


In [6]:
df_linhvuckhac=pd.read_csv("../data/data_tinhhuong/linhvuckhac_data.csv",encoding='utf-8')

In [7]:
df_linhvuckhac.head(5)

,Index,Name,Time,Question,Situation,Answer
0,0,Đào Long Sơn,14:49 28/08/2023,Thi tìm hiểu pháp luật có phải hoạt động văn h...,"Tôi xin hỏi, các địa phương tổ chức các cuộc t...","Bộ Văn hóa, Thể thao và Du lịch trả lời vấn đề..."
1,1,Nguyên Hạnh,23:37 28/06/2023,Tổ chức cuộc thi về ẩm thực có cần xin giấy phép?,Đơn vị tôi muốn tổ chức một cuộc thi về ẩm thự...,"Cục Văn hóa cơ sở, Bộ Văn hóa, Thể thao và Du ..."
2,2,Nguyễn Văn Tú,20:58 24/05/2023,Có quy định xét công nhận danh hiệu 'Tộc họ vă...,"Tôi muốn đăng ký xây dựng ""Tộc họ văn hoá"" của...","Cục Văn hóa cơ sở, Bộ Văn hóa, Thể thao và Du ..."
3,3,Đỗ Quốc Hưng,07:05 18/03/2023,"Có cần phần ""Kính gửi"" trong báo cáo không?","Nghị định số 30/2020/NĐ-CP quy định, ngoài tờ ...",Bộ Nội vụ trả lời vấn đề này như sau:Báo cáo l...
4,4,Lý Hoàng Long,09:34 17/03/2023,Tổ chức biểu diễn trên mạng có cần xin cấp phép?,Tôi dự định mở một cuộc biểu diễn trên nền tản...,"Cục Nghệ thuật biểu diễn, Bộ Văn hóa, Thể thao..."
